<a href="https://colab.research.google.com/github/amilyk/transformers-for-NLP/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 两种类型的语言建模任务
1. 因果语言模型（Causal language modeling，CLM）,类似于GPT或Decoder，前n个token预测下一个token（从左往右顺序生成）
2. 掩蔽语言建模（Masked language modeling，MLM），模型需要恢复输入中被"MASK"掉的一些字符，双向

# 1：因果语言模型（Causal Language Modeling，CLM）

获取到数据集中的所有文本，并在它们被分词后将它们连接起来,将在特定序列长度拆开，分成多个block，输入与标签相同。

# 加载数据集

In [1]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [2]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [3]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
show_random_elements(datasets["train"])

,text
0,"In 1981 , Wever began the season remaining with Fort Lauderdale . He had a 7 – 3 record and a 2 @.@ 00 ERA in 12 games before being promoted to the Nashville Sounds of the Southern League , the Yankees ' AA affiliate . With Nashville , he had a 5 – 2 record and 2 @.@ 05 ERA in nine appearances . Wever 's pitching coach in Nashville was Hall of Famer Hoyt Wilhelm . Wilhelm felt he had the ability but not the confidence to pitch in the majors , and spent his time in Nashville working on that aspect of Wever 's game . The following season , Weber was almost unanimously named to the Southern League All @-@ Star team , thanks to 11 wins and 116 strikeouts through the end of June . He improved to a 16 @-@ 6 record , a 2 @.@ 78 ERA , and 191 strikeouts , won the Southern League Pitcher of the Year award , and accomplished the pitcher 's Triple Crown , leading the league in wins , ERA , and strikeouts . He led Nashville to the Southern League championship , and right after doing so , the Yankees called Weber up to the major leagues . \n"
1,= = Route description = = \n
2,
3,"Meginness ( 1892 ) wrote that "" Plunkett 's Creek township , on account of its dashing mountain streams of pure water , has always been a favorite place for trout fishing . "" In 2007 , the Pennsylvania Fish and Boat Commission classified both Engle Run and the Noon Branch of Wolf Run as Class A Wild Trout Waters , defined as "" streams which support a population of naturally produced trout of sufficient size and abundance to support a long @-@ term and rewarding sport fishery . "" Barbours has been popular from early on with "" anglers seeking trout in the ' Sock and its tributaries "" , as well as with hunters after black bear , white @-@ tailed deer , and wild turkey in the surrounding forests . \n"
4,
5,"Survivor Series 1992 was attended by 17 @,@ 500 fans , the same number as the previous year . It drew more fans than any of the following three Survivor Series event would draw . The pay @-@ per @-@ view buyrate was 1 @.@ 4 , which means that 1 @.@ 4 percent of households to which the event was available purchased the pay @-@ per @-@ view . This was , to that point , the lowest buyrate in Survivor Series history and down more than one @-@ third from the previous year 's 2 @.@ 2 buyrate . The buyrate was higher than that of any of the following twelve Survivor Series events , however . \n"
6,"At the 2012 Consultative Group meeting of the Global Facility for Disaster Reduction and Recovery ( GFDRR ) , the Haitian delegation shared a "" bottom @-@ up "" approach to disaster reduction and management based on community integration and sustainable development with a group of experts from approximately 38 nations . \n"
7,"Fraser and Navarro claim that Eva Duarte had no knowledge of or interest in politics prior to meeting Perón . Therefore , she never argued with Perón or any of his inner circle , but merely absorbed what she heard . Juan Perón later claimed in his memoir that he purposefully selected Eva as his pupil , and set out to create in her a "" second I. "" Fraser and Navarro , however , suggest that Juan Perón allowed Eva Duarte such intimate exposure and knowledge of his inner circle because of his age : he was 48 and she was 24 when they met . He had come to politics late in life , and was therefore free of preconceived ideas of how his political career should be conducted , and he was willing to accept whatever aid she offered him . \n"
8,"Codium fragile ssp. fragile ( formerly reported as ssp. tomentosum ) , now well established . \n"
9,= Temple of Eshmun = \n


# 预处理数据集

In [5]:
model_checkpoint = "distilgpt2"

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
#batch=True和4个进程来加速预处理
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

将所有文本连接，分割成特定block_size的小块，标签将与输入相同

In [9]:
# block_size = tokenizer.model_max_length
block_size = 128

In [10]:
def group_texts(examples):
    # 拼接所有文本
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # 我们将余数对应的部分去掉。但如果模型支持的话，可以添加padding，您可以根据需要定制此部件。
    total_length = (total_length // block_size) * block_size
    # 通过max_len进行分割。
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [12]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])
#内容不再是单一文本，block_size连续字符块，可能跨越了几个原始文本

' game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Oz'

In [13]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:

import importlib.util
# import importlib_metadata
a = importlib.util.find_spec("torch") is not None
print(a)
# _torch_version = importlib_metadata.version("torch")
# print(_torch_version)

True


# 微调模型

In [15]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    "test-clm",
    # evaluation_strategy = "epoch",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xunkangzju (xunkangzju-zj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.719500,3.637664


# 2：掩蔽语言模型（Mask Language Modeling，MLM）

# 预处理数据集（相同dataset）

In [ ]:
model_checkpoint = "distilroberta-base"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
#若datasets是单独的句子，不需要分隔成对应block
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
# 批处理数据的时候，被随机"MASK"，mask概率为mlm_probability
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# 微调模型

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
training_args = TrainingArguments(
    "test-mlm",
    # evaluation_strategy = "epoch",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")